<a href="https://colab.research.google.com/github/wayne0git/ml_cv_basics/blob/master/tflite/tflite_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensorflow Lite Basics
Ref - https://learning.edx.org/course/course-v1:HarvardX+TinyML2+3T2020

In [1]:
import tensorflow as tf # 2.4.0
import numpy as np   # 1.19.4

### Prepare simple Tensorflow model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Linear regression model
model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

# Train
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

In [3]:
# Check result
print(model.predict([10.0]))
print("Here is what I learned: {}".format(model.layers[0].get_weights()))

[[18.977861]]
Here is what I learned: [array([[1.9967912]], dtype=float32), array([-0.9900514], dtype=float32)]


In [4]:
# Save Tensorflow Model
# *.pb / assets / variables
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


### Tensorflow Lite Converter

In [22]:
# Init the converter
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

In [23]:
# 1. Convert Options (No Optimization)
converter.optimizations = []

# 2. Convert Options (Default Optimization)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

# 3. Quantization based on tensorflow_datasets object
#def representative_data_gen():                          # Uncomment the following lines for Model 3
#    for input_value, _ in test_batches.take(100):
#        yield [input_value]
# converter.representative_dataset = representative_data_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

In [24]:
# Convert the model
tflite_model = converter.convert()

In [25]:
# Save the TFLite model
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model) # Return the total byte number of the model

944

### Tensorflow Lite Interpreter

In [ ]:
# Load TFLite model (From File)
interpreter = tf.lite.Interpreter(model_path='model.tflite')

# Load TFLite model (From convert model)
# interpreter = tf.lite.Interpreter(model_content=tflite_model)

In [ ]:
# Allocate tensors.
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details) # list[dict]
print(output_details) # list[dict]

[{'name': 'serving_default_dense_1_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
# Inference
interpreter.set_tensor(input_details[0]['index'], np.array([[10.0]], dtype=np.float32))
interpreter.invoke()
interpreter.get_tensor(output_details[0]['index'])

array([[18.987093]], dtype=float32)